In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import random
from collections import Counter

df_all = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')

##reading all STATS from one excel##
excel_filename = "/Users/roumyadas/Desktop/IPL_Simulation/Season_02/STATS_S02.xlsx"

pts_table = pd.read_excel(excel_filename, sheet_name='POINTS_TABLE')

matches_list = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_02')

matches_list = matches_list[['Date','Team One','Team Two']]
mts_done = pts_table['matches_played'].sum()/2
matches_list_mod = matches_list[int(mts_done):]
matches_list_mod['winner'] = ''

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_22062/2637981220.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_list_mod['winner'] = ''


In [2]:
q_master_list = []

In [3]:
%%time

# Vectorized simulation
for _ in range(10**4):
    # Randomly assign winners
    matches_list_mod["winner"] = np.where(
        np.random.rand(len(matches_list_mod)) < 0.5, 
        matches_list_mod["Team One"], 
        matches_list_mod["Team Two"]
    )

    # Count wins and calculate points
    sim_win = matches_list_mod["winner"].value_counts().reset_index()
    sim_win.columns = ["team", "wins"]
    sim_win["pts"] = sim_win["wins"] * 2

    # Merge results with original points table
    pts_table_mod = pts_table.merge(sim_win, on="team", how="left").fillna(0)
    pts_table_mod["pts_final"] = pts_table_mod["points"] + pts_table_mod["pts"]
    pts_table_mod["NRR_final"] = pts_table_mod["NRR"] + np.random.normal(0, 0.1, len(pts_table_mod))

    # Sort by points and NRR
    pts_table_mod = pts_table_mod.sort_values(by=["pts_final", "NRR_final"], ascending=False)

    # Get top 4 teams
    q_master_list.append(pts_table_mod["team"].head(4).tolist())

flat_list = [item for sublist in q_master_list for item in sublist]
counts = Counter(flat_list)

df = pd.DataFrame(counts.items(), columns=["String", "Count"])

df = df.sort_values(by="Count", ascending=False).reset_index(drop=True)
df['prob_qualification'] = 100*df['Count']/len(q_master_list)

<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 26.8 s, sys: 154 ms, total: 27 s
Wall time: 27.1 s


In [4]:
print("Qualification Probabilities ::")
for idx, row in df[['String','prob_qualification']].iterrows():
    print(f"{row['String']} -- {row['prob_qualification']}")


Qualification Probabilities ::
SRH -- 100.0
DC -- 100.0
RR -- 100.0
CSK -- 57.48
MI -- 42.52
